## Install Grounding DINO 🦕

In [1]:
import os
HOME = os.getcwd()

In [1]:
import os
HOME = os.getcwd()

%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
!pip install -q -e .
!pip install -q roboflow

!mkdir {HOME}/weights
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth
!pip install -q loguru
!pip install torchvision

/Users/tarik.setia/lab/cv-benchmark
Cloning into 'GroundingDINO'...


/Users/tarik.setia/lab/cv-benchmark/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


remote: Enumerating objects: 443, done.
remote: Counting objects: 100% (211/211), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 443 (delta 156), reused 137 (delta 128), pack-reused 232
Receiving objects: 100% (443/443), 12.86 MiB | 1.85 MiB/s, done.
Resolving deltas: 100% (228/228), done.
/Users/tarik.setia/lab/cv-benchmark/GroundingDINO
Reason for being yanked: deprecated, use 4.8.0.76
/Users/tarik.setia/lab/cv-benchmark/weights


In [2]:
!python -m pip install -q pillow
!python -m pip install -q loguru
!python -m pip install -q pydantic
!python -m pip install -q pandas

## Restart Notebook at this point

In [1]:
import os
HOME = os.getcwd()

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

/Users/tarik.setia/lab/cv-benchmark/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py ; exist: True
/Users/tarik.setia/lab/cv-benchmark/weights/groundingdino_swint_ogc.pth ; exist: True


In [2]:
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cpu'

In [4]:
from groundingdino.util.inference import Model, predict, preprocess_caption, load_model
from groundingdino.util.utils import get_phrases_from_posmap

model = load_model(CONFIG_PATH, WEIGHTS_PATH, DEVICE)
model = model.to(DEVICE)

final text_encoder_type: bert-base-uncased


In [6]:
import numpy as np
import torch
from PIL import Image

import groundingdino.datasets.transforms as T

def preprocess_frame(frame: np.array) -> torch.Tensor:
    transform = T.Compose(
        [
            T.RandomResize([800], max_size=1333),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
    image_source = Image.fromarray(frame.astype("uint8"))
    image_transformed, _ = transform(image_source, None)
    return image_transformed

In [7]:
from typing import List, Tuple
import bisect

def predict(
        model,
        image: torch.Tensor,
        caption: str,
        box_threshold: float,
        text_threshold: float,
        device: str = "cuda",
        remove_combined: bool = False
) -> Tuple[torch.Tensor, torch.Tensor, List[str]]:
    caption = preprocess_caption(caption=caption)


    with torch.no_grad():
        outputs = model(image[None], captions=[caption])

    prediction_logits = outputs["pred_logits"].cpu().sigmoid()[0]  # prediction_logits.shape = (nq, 256)
    prediction_boxes = outputs["pred_boxes"].cpu()[0]  # prediction_boxes.shape = (nq, 4)

    mask = prediction_logits.max(dim=1)[0] > box_threshold
    logits = prediction_logits[mask]  # logits.shape = (n, 256)
    boxes = prediction_boxes[mask]  # boxes.shape = (n, 4)

    tokenizer = model.tokenizer
    tokenized = tokenizer(caption)
    
    if remove_combined:
        sep_idx = [i for i in range(len(tokenized['input_ids'])) if tokenized['input_ids'][i] in [101, 102, 1012]]
        
        phrases = []
        for logit in logits:
            max_idx = logit.argmax()
            insert_idx = bisect.bisect_left(sep_idx, max_idx)
            right_idx = sep_idx[insert_idx]
            left_idx = sep_idx[insert_idx - 1]
            phrases.append(get_phrases_from_posmap(logit > text_threshold, tokenized, tokenizer, left_idx, right_idx).replace('.', ''))
    else:
        phrases = [
            get_phrases_from_posmap(logit > text_threshold, tokenized, tokenizer).replace('.', '')
            for logit
            in logits
        ]

    return boxes, logits.max(dim=1)[0], phrases


In [19]:
import time
from utils.protocols import GDino
from utils.utils import convert_model_detection, get_file_name, get_gpu_name
from utils.video import read_video
from loguru import logger
from datetime import datetime

TEXT = "Face."
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25
MODEL="grounding-dino-tiny-github"
BASE_DIR="experiments/gdino"

def run_model(frames):
    results = {}
    for frame_id, frame in frames.items():
        results[frame_id] = predict(
            model=model,
            image=frame,
            caption=TEXT,
            box_threshold=BOX_TRESHOLD,
            text_threshold=TEXT_TRESHOLD,
            device=DEVICE
        )
    return results

def process_video(video, frames=[]):
    start_time = datetime.now()
    source_h, source_w = None, None
    
    frames_ = {}
    start = time.time()
    for frame_id, frame in read_video(video):
        frames_[frame_id] = Model.preprocess_image(frame).to(DEVICE)
        if not source_h:
            source_h, source_w, _ = frame.shape

    pre_processing_time = time.time() - start
    
    frames_ = {k: frames_[k] for k in list(frames_.keys())[:3]}
    n_frames = len(frames_)
    
    start = time.time()
    outputs = run_model(frames_)
    inference_time = time.time() - start
    
    results = {}
    start = time.time()
    
    for frame_id, output in outputs.items():    
        boxes, logits, phrases = output
        results[frame_id] = convert_model_detection({"boxes": boxes, "labels":phrases, "scores": logits})
    post_processing_time = time.time() - start
    
    end_time = datetime.now()
    
    
    exp =  GDino(
        model=MODEL,
        gpu=get_gpu_name(),
        video_file=video,
        frames=None if not frames else frames,
        n_frames=n_frames,
        
        pre_processing_time=pre_processing_time,
        inference_time=inference_time,
        post_processing_time=post_processing_time,
        video_processing_time=(end_time-start_time).seconds,
       
        start_time=start_time.isoformat(),
        end_time=end_time.isoformat(),
        record_file=get_file_name(BASE_DIR, start, MODEL, video),
        
        data=results,
        prompt=TEXT,
    )
    exp.save()
    exp.log()
    torch.cuda.empty_cache()
    return exp

In [20]:
vid_hd = process_video("data/720.mp4")

In [21]:
vid_hd

{0: [Detection(box=[0.4578925371170044, 0.27158764004707336, 0.0287613682448864, 0.12781035900115967], score=0.4847736060619354, label='face'),
  Detection(box=[0.505282461643219, 0.14036715030670166, 0.019930114969611168, 0.05196128785610199], score=0.36129269003868103, label='face'),
  Detection(box=[0.3546461760997772, 0.19461601972579956, 0.040837787091732025, 0.15716183185577393], score=0.3599214255809784, label='face')],
 1: [Detection(box=[0.44923439621925354, 0.28405627608299255, 0.0302472785115242, 0.12504100799560547], score=0.5427340269088745, label='face'),
  Detection(box=[0.7070327401161194, 0.375203013420105, 0.021796727553009987, 0.09171603620052338], score=0.4986567795276642, label='face'),
  Detection(box=[0.353489488363266, 0.21009130775928497, 0.032213836908340454, 0.11471692472696304], score=0.40450069308280945, label='face')],
 2: [Detection(box=[0.684285581111908, 0.27359580993652344, 0.059744276106357574, 0.2832659184932709], score=0.4769657254219055, label='fac

In [16]:
results = []
for i in range(5):
    vid_hd = process_video("data/720.mp4")
    results.append(vid_hd)
    vid_fhd = process_video("data/1080.mp4")
    results.append(vid_fhd)

2024-06-11 10:54:17.760 | INFO     | utils.protocols:log:88 - data/720.mp4 | frames=283 | delta=54.0480523109436 | fps=5.236081373883262
2024-06-11 10:55:16.183 | INFO     | utils.protocols:log:88 - data/1080.mp4 | frames=283 | delta=58.413251876831055 | fps=4.844791051810089
2024-06-11 10:56:11.900 | INFO     | utils.protocols:log:88 - data/720.mp4 | frames=283 | delta=55.70707559585571 | fps=5.080144613102856
2024-06-11 10:57:09.442 | INFO     | utils.protocols:log:88 - data/1080.mp4 | frames=283 | delta=57.531291246414185 | fps=4.9190621984803595
2024-06-11 10:58:04.724 | INFO     | utils.protocols:log:88 - data/720.mp4 | frames=283 | delta=55.27209115028381 | fps=5.120124715935356
2024-06-11 10:59:04.258 | INFO     | utils.protocols:log:88 - data/1080.mp4 | frames=283 | delta=59.52292799949646 | fps=4.754470411845231
2024-06-11 10:59:57.977 | INFO     | utils.protocols:log:88 - data/720.mp4 | frames=283 | delta=53.70751929283142 | fps=5.269280795804197
2024-06-11 11:00:55.173 | INF

In [17]:
columns = results[0].columns
rows = [result.row for result in results]

In [18]:
import pandas as pd

df = pd.DataFrame(rows, columns=columns)
df.head()

,model,gpu,file,batch_size,n_frames,processing_time,fps,start_time,end_time,result_file,prompt,data
0,grounding-dino-tiny-github,Tesla V100-SXM2-16GB,data/720.mp4,1,283,54.048052,5.236081,2024-06-11T10:53:23.702729,2024-06-11T10:54:17.750796,experiments/gdino/17181032037027397-grounding-...,Face.,"{0: [box=[0.45709970593452454, 0.2791376411914..."
1,grounding-dino-tiny-github,Tesla V100-SXM2-16GB,data/1080.mp4,1,283,58.413252,4.844791,2024-06-11T10:54:17.761880,2024-06-11T10:55:16.175152,experiments/gdino/17181032577618957-grounding-...,Face.,"{0: [box=[0.3344019651412964, 0.18687993288040..."
2,grounding-dino-tiny-github,Tesla V100-SXM2-16GB,data/720.mp4,1,283,55.707076,5.080145,2024-06-11T10:55:16.185075,2024-06-11T10:56:11.892169,experiments/gdino/17181033161850908-grounding-...,Face.,"{0: [box=[0.45709970593452454, 0.2791376411914..."
3,grounding-dino-tiny-github,Tesla V100-SXM2-16GB,data/1080.mp4,1,283,57.531291,4.919062,2024-06-11T10:56:11.902434,2024-06-11T10:57:09.433739,experiments/gdino/17181033719024425-grounding-...,Face.,"{0: [box=[0.3344019651412964, 0.18687993288040..."
4,grounding-dino-tiny-github,Tesla V100-SXM2-16GB,data/720.mp4,1,283,55.272091,5.120125,2024-06-11T10:57:09.443876,2024-06-11T10:58:04.715988,experiments/gdino/17181034294438922-grounding-...,Face.,"{0: [box=[0.45709970593452454, 0.2791376411914..."


In [19]:
from datetime import datetime
now = datetime.now()
csv_file = f"zz-{MODEL}-{get_gpu_name()}-{now.day}-{now.hour}-{now.minute}.csv"
df.to_csv(csv_file)